# Two Bar Truss 

This is a rewrite of the two bar truss genetic optimization using a class. Much neater, easier to use and faster (rewrote a few of the functions to use vectorized numpy operations)

In [1]:
import numpy as np
import genops as op

### The complete class definition

In [5]:
class TwoBar:
    def __init__(self):
        # GA settings
        self.numiter = 300
        self.pop_sz = 50
        self.ind_sz = 2
        
        # Two bar truss specifications
        self.density = 0.3
        self.modulus = 30000
        self.load = 66
        self.width = 60
        self.thickness = 0.15
        
        # Results stored here
        self.result = -1
        self.height = -1
        self.diameter = -1
        self.weight = -1
    
    def run(self):
        """ Run the ga """
        pop = self.generate()
    
        for i in range(0,self.numiter):
            new_pop = self.tournament_select(pop)
            pop = self.best_population(pop,new_pop)
        self.result = pop
        self.height = pop[0,1]*100+1
        self.diameter = pop[0,2]*100+1
        self.weight = pop[0,0]
        
    def two_bar_truss(self,density, modulus, load, width, thickness, height, diameter):
        """ Calculate the weight, stress, buckling and deflection"""
        length = ((width/2.0)**2.0 + height**2)**0.5
        area = np.pi * diameter * thickness
        iovera = (diameter**2.0 + thickness**2.0)/8.0
        weight = 2.0 * density * length * area
        stress = load * length/(2.0 * area * height)
        buckling = np.pi**2.0 * modulus * iovera/(length**2.0)
        deflection = load * length**3.0/(2.0 * modulus * area * height**2.0)
        return(weight, stress, buckling, deflection)
    
    # SBX Crossover
    def crossover(self,parent1,parent2,eta,low,up):
        """ SBX real crossover """
        child1 = np.zeros(len(parent1))
        child2 = np.zeros(len(parent2))
        for i in range(0,len(parent1)):
            child1[i], child2[i] = op.sbx_real(parent1[i],parent2[i],eta,low,up)
        return child1,child2

    # Real mutation
    def mutate(self,individual, prob, eta, low, up):
        """ Real mutation of individuals """
        for i in range(0,len(individual)):
            individual[i] = op.mutate_real(individual[i],prob, eta, low, up)
        return individual
    
    # Return the best individual of two selected at random
    def get_random_best_individual(self,population):
        sz = population.shape[0]
        rn = np.random.randint(0,sz,2) # Generate 2 random numbers between 0 and size of population
        t1 = population[rn[0]] # Grab random individual
        t2 = population[rn[1]] # Grab another random individual
        f1 = t1[0] # Grab fitness from array (fitness is first entry in array)
        f2 = t2[0] # Grab fitness from array (fitness is first entry in array)
        if f1 < f2:
            return t1
        else:
            return t2

    # Tournament selection
    def tournament_select(self,population):
        new_pop = np.zeros((self.pop_sz,self.ind_sz+1))
        for i in range(0,self.pop_sz,2):
            best1 = self.get_random_best_individual(population)
            best2 = self.get_random_best_individual(population)
            child1, child2 = self.crossover(best1, best2,1,0,1)
            child1 = self.mutate(child1,0.1,1,0,1)
            child2 = self.mutate(child2,0.1,1,0,1)
            new_pop[i] = child1
            new_pop[i+1] = child2
        return new_pop
    
    # Add together new and old populations and grab top 50 results
    # sorted by lowest first
    def best_population(self,pop,new_pop):
        final = np.vstack((pop,new_pop)) # stack both arrays
        final[:,0] = self.fitness(final[:,1],final[:,2]) # Calculate fitness
        final = final[np.argsort(final[:,0])] # Sort ascending
        final = final[:self.pop_sz,:] # Grab top population
        return final
    
    # Calculate fitness of population
    def fitness(self,height,diameter):
        w,s,b,d = self.two_bar_truss(self.density,self.modulus,self.load
                                    ,self.width,self.thickness,height*100+1,diameter*100+1) 
        # If any value exceeds constraints weight result
        w = np.where(s>100,w*2,w)
        w = np.where(d>0.25,w*2,w)
        w = np.where((s-b)>0.0,w*2,w)
        return w
    
    def find_valid_individual(self):
        for i in range(0,1000): 
            temp = np.random.rand(2)
            w,s,b,d = self.two_bar_truss(self.density,self.modulus,self.load
                                    ,self.width,self.thickness,(temp[0]*100)+1,(temp[1]*100)+1) 
            if s < 100.0 and d < 0.25 and s-b < 0.0:
                return w,temp[0],temp[1] # Return fitness, height, diameter

    # Generate a random population
    def generate(self):
        pop = np.zeros((self.pop_sz,self.ind_sz+1))
        for i in range (0, self.pop_sz):
            pop[i] = self.find_valid_individual()
        return pop

### Setup the problem

In [3]:
DENSITY = 0.3
MODULUS = 30000
LOAD = 66
WIDTH = 60
THICKNESS = 0.15

test = TwoBar()
#test.load=120

### Run the optimization

In [4]:
%time test.run()

Wall time: 239 ms


### Print the results

In [ ]:
print("Weight: {0:.4f} Height: {1:.4f} Diameter: {2:.4f}".format(test.weight, test.height, test.diameter))